# Overview
The tool serves to let you create task files from CSVs and zip files that you upload through the browser

In [1]:
import ipywidgets as ipw
import pandas as pd
import json, io, os, tempfile
import fileupload as fu
from IPython.display import display, FileLink

def upload_as_file_widget(callback=None):
    """Create an upload files button that creates a temporary file and calls a function with the path.
    """
    _upload_widget = fu.FileUploadWidget()
    
    def _virtual_file(change):
        file_ext = os.path.splitext(change['owner'].filename)[-1]
        print('Uploaded `{}`'.format(change['owner'].filename))
        if callback is not None:
            with tempfile.NamedTemporaryFile(suffix=file_ext) as f:
                f.write(change['owner'].data)
                callback(f.name)

    _upload_widget.observe(_virtual_file, names='data')
    
    return _upload_widget
def make_task(in_df, 
              image_path='Image Index', 
              output_labels='Finding Labels', 
              base_image_directory = 'sample_data'):
    return {
        'google_forms': {'form_url': 'https://docs.google.com/forms/d/e/1FAIpQLSfBmvqCVeDA7IZP2_mw_HZ0OTgDk2a0JN4VlY5KScECWC-_yw/viewform', 
        'sheet_url': 'https://docs.google.com/spreadsheets/d/1T02tRhe3IUUHYsMchc7hmH8nVI3uR0GffdX1PNxKIZA/edit?usp=sharing'
                },
        'dataset': {
            'image_path': image_path, # column name
            'output_labels': output_labels, # column name
            'dataframe': in_df.to_dict(),
            'base_image_directory': base_image_directory # path
        }
    }
def save_task(annotation_task, out_path='task.json'):
    with open(out_path, 'w') as f:
        json.dump(annotation_task, f)
    return out_path

/srv/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/srv/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Instructions
Load a CSV file and select the columns for the image path, labels and the name of the directory where the images are located

In [4]:
def _load_csv_app(in_path):
    """
    A callback to create an app from an uploaded CSV file
    >>> _load_csv_app('sample_data/dataset_overview.csv')
    """
    ds_df = pd.read_csv(in_path)
    table_viewer = ipw.HTML(value=ds_df.sample(3).T.to_html(), layout = ipw.Layout(width="45%"))
    image_path_widget = ipw.Dropdown(
        options=ds_df.columns,
        value=ds_df.columns[0],
        description='Image Path Column:',
        disabled=False
    )
    output_labels_widget = ipw.Dropdown(
        options=ds_df.columns,
        value=ds_df.columns[0],
        description='Label Column:',
        disabled=False
    )
    all_dir_list = [p for p, _, _ in os.walk('.') if os.path.isdir(p) and not any([k.startswith('.') and len(k)>1 for k in p.split('/')])]
    base_image_directory_widget = ipw.Select(
        options=all_dir_list,
        value=None,
        rows=5,
        description='Local Image Folder:',
        disabled=False
    )
    def _create_task(btn):
        c_task = make_task(ds_df, 
                  image_path = image_path_widget.value,
                  output_labels = output_labels_widget.value,
                  base_image_directory = base_image_directory_widget.value
                 )
        display(FileLink(save_task(c_task)))
    
    create_but = ipw.Button(description='Create Task')
    create_but.on_click(_create_task)
    controls = ipw.VBox([image_path_widget, output_labels_widget,
                                              base_image_directory_widget, create_but])
    out_widget = ipw.HBox([controls,
                           table_viewer])
    display(out_widget)
    return out_widget

In [5]:
upload_as_file_widget(_load_csv_app)

FileUploadWidget(label='Browse', _dom_classes=('widget_item', 'btn-group'))

Uploaded `data_summary.csv`


/home/jovyan/task.json